In [ ]:
srun -G1 --partition dgx --pty bash -c "source /data/ai_club/team_3_2024-25/team3-env-dgx/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

srun -G1 --partition dgx --pty bash

MAKE_ARGS="-DGGML_CUDA=on" pip install --verbose llama-cpp-python

srun -G1 --partition dgx --pty bash
source /data/ai_club/team_3_2024-25/team3-env-h100/bin/activate

srun -G1 --partition dgxh100 --pty bash -c "source /data/ai_club/team_3_2024-25/team3-env-h100/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

srun -G4 --pty bash -c "source /data/ai_club/team_3_2024-25/team3-env-py312-glibc/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

# singularity exec --nv  -B/data:/data,/home:/home /data/containers/msoe-pytorch-24.05-py3.sif \

srun --job-name=LLTM -G3 --partition dgx --pty \
    bash -c "source /data/ai_club/team_3_2024-25/team3-env-dgx/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

srun --job-name=LLTM -G4 --pty \
    bash -c "source /data/ai_club/team_3_2024-25/team3-env-finetune/bin/activate; \
    hostname; \
    jupyter notebook \
        --ServerApp.root_dir=$(pwd) \
        --ServerApp.password='' \
        --ServerApp.open_browser=False \
        --ServerApp.allow_origin='*' \
        --ServerApp.allow_remote_access=True \
        --ServerApp.port=14321 \
        --ServerApp.ip='*'
"

In [1]:
%cd Documents/aiClub/lltm/LLTM/llm_core

/home/grall/Documents/aiClub/lltm/LLTM/llm_core


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

KeyboardInterrupt: 

In [2]:
# with open('../.env', 'r') as f:
#     TOKEN = f.read()

model_id = "meta-llama/Llama-3.3-70B-Instruct"

bnb = BitsAndBytesConfig(
    load_in_8bit=True,
    # bnb_8bit_use_double_quant=True,
    # bnb_8bit_quant_type="nf8",
    # bnb_8bit_compute_dtype=torch.bfloat16,

    llm_int8_enable_fp32_cpu_offload=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    low_cpu_mem_usage=True,
    # attn_implementation="flash_attention_2",
    torch_dtype=torch.float16,
    quantization_config=bnb,
)


2025-03-19 17:23:44.657366: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-19 17:23:44.693366: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742419424.714700  153106 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742419424.719099  153106 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-19 17:23:44.735703: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [3]:
!nvidia-smi

Wed Mar 19 17:26:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:60:00.0 Off |                    0 |
| N/A   52C    P0             29W /   70W |   11935MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
model.hf_device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 0,
 'model.layers.7': 0,
 'model.layers.8': 0,
 'model.layers.9': 0,
 'model.layers.10': 0,
 'model.layers.11': 0,
 'model.layers.12': 1,
 'model.layers.13': 1,
 'model.layers.14': 1,
 'model.layers.15': 1,
 'model.layers.16': 1,
 'model.layers.17': 1,
 'model.layers.18': 1,
 'model.layers.19': 1,
 'model.layers.20': 1,
 'model.layers.21': 1,
 'model.layers.22': 1,
 'model.layers.23': 1,
 'model.layers.24': 1,
 'model.layers.25': 1,
 'model.layers.26': 1,
 'model.layers.27': 1,
 'model.layers.28': 2,
 'model.layers.29': 2,
 'model.layers.30': 2,
 'model.layers.31': 2,
 'model.layers.32': 2,
 'model.layers.33': 2,
 'model.layers.34': 2,
 'model.layers.35': 2,
 'model.layers.36': 2,
 'model.layers.37': 2,
 'model.layers.38': 2,
 'model.layers.39': 2,
 'model.layers.40': 2,
 'model.layers.41': 2,
 'model.layers.42'

In [5]:
inputs = tokenizer.encode('''
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a Finnish language teacher.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the mistake in "Tama on minun koira"<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''.strip()+'\n\n', return_tensors='pt').to('cuda')

In [6]:
outputs = model.generate(
    inputs,
    max_new_tokens=1,
    attention_mask=torch.ones(inputs.shape).to('cuda'),
    # temperature=0.0,
    # do_sample=True
)
# tok = outputs.logits[0,-1].argmax()
# print(tokenizer.decode(tok), end='')
# inputs += tok

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


OutOfMemoryError: CUDA out of memory. Tried to allocate 448.00 MiB. GPU 0 has a total capacity of 14.57 GiB of which 332.75 MiB is free. Including non-PyTorch memory, this process has 14.24 GiB memory in use. Of the allocated memory 14.07 GiB is allocated by PyTorch, and 37.86 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [35]:
tokenizer.decode(outputs[0])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a Finnish language teacher.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the mistake in "Tama on minun koira"<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nHyvä kysymys! (Good question!)\n\nThe mistake in "Tama on minun koira" is the use of "minun" instead of "minun" should be in the genitive case, which is "minun" but the correct word order and possessive suffix would make it "Minun koira" or "Minulla on koira" or simply "Tämä on minun koirani".\n\nHowever, the most common mistake is the word "T'

In [17]:
tokenizer.decode(outputs.logits.argmax(dim=-1)[0])

'QuestionQuestion \n\n\n###ting Knowledge Date:  2023\nMost Date: February29 February 2024\nThe can now large person teacher, I<|start_header_id|>assistant<|end_header_id|>\n\nHe you teach the grammar \n<|start_header_id|>assistant<|end_header_id|>\n\nT'

In [7]:
outputs = model(
    inputs,
    max_new_tokens=1,
    # attention_mask=torch.ones(inputs.shape).to('cuda'),
    temperature=0.5,
    do_sample=True
)
# tok = outputs.logits[0,-1].argmax()
# print(tokenizer.decode(tok), end='')
# inputs += tok

/data/ai_club/team_3_2024-25/team3-env-dgx/lib/python3.12/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


In [42]:
tokenizer.decode(outputs[0])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nYou are a Finnish language teacher.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nDo you know Finnish?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nKyllä! (Yes, I do'

---

In [1]:
from llama_cpp import Llama

In [4]:
(4096 + 8192)/2

6144.0

In [2]:
llm = Llama(
    n_ctx=4096,
    model_path='/data/ai_club/llms/Llama-3.3-70B-Instruct-Q6_K_L-00001-of-00002.gguf',

    n_gpu_layers=-1, verbose=True,
    flash_attn=True
    # embedding=True
)

llama_model_loader: additional 1 GGUFs metadata loaded.
llama_model_loader: loaded meta data with 43 key-value pairs and 724 tensors from /data/ai_club/llms/Llama-3.3-70B-Instruct-Q6_K_L-00001-of-00002.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Llama 3.3 70B Instruct
llama_model_loader: - kv   3:                           general.finetune str              = Instruct
llama_model_loader: - kv   4:                           general.basename str              = Llama-3.3
llama_model_loader: - kv   5:                         general.size_label str              = 70B
llama_model_loader: - kv   6:                            g

llama_model_loader: - type q6_K:  560 tensors
llm_load_vocab: special tokens cache size = 256
llm_load_vocab: token to piece cache size = 0.7999 MB
llm_load_print_meta: format           = GGUF V3 (latest)
llm_load_print_meta: arch             = llama
llm_load_print_meta: vocab type       = BPE
llm_load_print_meta: n_vocab          = 128256
llm_load_print_meta: n_merges         = 280147
llm_load_print_meta: vocab_only       = 0
llm_load_print_meta: n_ctx_train      = 131072
llm_load_print_meta: n_embd           = 8192
llm_load_print_meta: n_layer          = 80
llm_load_print_meta: n_head           = 64
llm_load_print_meta: n_head_kv        = 8
llm_load_print_meta: n_rot            = 128
llm_load_print_meta: n_swa            = 0
llm_load_print_meta: n_embd_head_k    = 128
llm_load_print_meta: n_embd_head_v    = 128
llm_load_print_meta: n_gqa            = 8
llm_load_print_meta: n_embd_k_gqa     = 1024
llm_load_print_meta: n_embd_v_gqa     = 1024
llm_load_print_meta: f_norm_eps       = 0.0

In [11]:
# input_toks = llm.tokenize(
# '''
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>

# You are a Finnish language teacher for formal Finnish.<|eot_id|><|start_header_id|>user<|end_header_id|>

# What are the mistakes in "Tama on minun koira"? Give me: [correct version] [breif list of mistakes].<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# '''.strip().encode(), special=True, add_bos=False
# )

input_toks = llm.tokenize(
'''
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a Finnish language teacher for formal Finnish.<|eot_id|><|start_header_id|>user<|end_header_id|>

Hei. Kuka sinä olet?<|eot_id|><|start_header_id|>assistant<|end_header_id|>
'''.strip().encode(), special=True, add_bos=False
)

# input_toks = llm.tokenize(
# '''
# <|begin_of_text|><|start_header_id|>system<|end_header_id|>

# You are a Finnish language teacher.<|eot_id|><|start_header_id|>user<|end_header_id|>

# Give me the broad IPA spelling for hyvä and breifly explain each phoneme.<|eot_id|><|start_header_id|>assistant<|end_header_id|>
# '''.strip().encode()
# )

In [12]:
output = llm(
    input_toks,
    max_tokens=200,
    stream=True,
    temperature=0
)

In [13]:
for tok in output:
    tok = tok['choices'][0]['text']
    if tok == '<|eot_id|>':
        print('<DONE>')
        break
    print(tok, end='')

Llama.generate: 20 prefix-match hit, remaining 14 prompt tokens to eval




Hei! Olen suomen kielen opettaja. Opetan formaalia suomea, eli virallista, kirjallista suomea. Miten voinkin auttaa sinua suomen kielen opiskelussa?

llama_perf_context_print:        load time =    1071.30 ms
llama_perf_context_print: prompt eval time =       0.00 ms /    14 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    57 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   20810.93 ms /    71 tokens


---

### Python Env Instructions

```
    Start a VSCode server on the Rosie dashboard.

    Open the command palette (F1 or Ctrl+Shift+P).

    Search for and select `Python: Select Interpreter`

    Choose the first option: `Enter interpreter path...`

    Choose the first option again: `Find...`

    Navigate to `/data/ai_club/team_3_2024-25/team3-env-py312-glibc/bin/python`

    Ignore all errors that may pop up (they shouldn't matter)

    In the upper-right corner, in this notebook toolbar, select a kernel. < - - - - - - <

    Choose `select another kernel`                                                      |

    Choose `Python Environments`                                                        |

    Choose `team3-env-py312-glibc`                                                      |

    Now you can run code. In the future you should only have to start from here > - - - ^
```

In [1]:
from llama_cpp import Llama, LogitsProcessorList
import numpy as np

In [2]:
!ls /data/ai_club/llms -lah # see llms you can use. Bigger models need more GPUs

total 89G
drwxrwsr-x  2 grall    ai_club 4.0K Apr  2  2024 .
drwxrws--- 39 sysadmin ai_club 4.0K Oct 25 16:15 ..
-rwxrwxr-x  1 grall    grall   6.4G Oct 29  2023 ggml-model-f16.gguf
-rwxrwxr-x  1 grall    grall   4.5G Sep  4  2023 llama-2-7b-chat.Q5_K_M.gguf
-rwxrwxr-x  1 grall    grall   7.2G Feb 19  2024 mistral-7b-instruct-v0.2.Q8_0.gguf
-rwxrwxr-x  1 grall    grall    36G Apr  2  2024 mixtral-8x7b-instruct-v0.1.Q6_K.gguf
-rwxrwxr-x  1 grall    grall    35G Apr  2  2024 yi-34b.Q8_0.gguf


In [2]:
llm = Llama(
    model_path='/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
    n_gpu_layers=-1, # put everything on GPU
    verbose=1,
    n_ctx=1000, # 4000
    logits_all=True
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7: 

In [13]:
s = llm('this is', max_tokens=10, stream=True)

In [18]:
llm.tokenize(bytes('this is', 'utf-8'))

[1, 456, 349]

In [14]:
t = next(s)

Llama.generate: 2 prefix-match hit, remaining 1 prompt tokens to eval


In [15]:
t

{'id': 'cmpl-d6402ff9-1ad9-4495-8476-609e6024022f',
 'object': 'text_completion',
 'created': 1733438946,
 'model': '/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
 'choices': [{'text': ' a',
   'index': 0,
   'logprobs': None,
   'finish_reason': None}]}

In [12]:
t['choices'][0]['text']

' first'

---

In [10]:
llm.tokenize(b'test', add_bos=False) # some words are multiple tokens

[1369]

---

In [29]:
resp = llm(
    "this",
    max_tokens=1, # change this!
    temperature=0 # change this!
)
resp

Llama.generate: 1 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     176.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /     1 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =      55.74 ms /     2 tokens


{'id': 'cmpl-6ed95800-93a8-404a-a731-d7177b1369e4',
 'object': 'text_completion',
 'created': 1730403009,
 'model': '/data/ai_club/llms/mistral-7b-instruct-v0.2.Q8_0.gguf',
 'choices': [{'text': ' is',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 2, 'completion_tokens': 1, 'total_tokens': 3}}

In [30]:
print(
    resp['choices'][0]['text']
)

 is


---

In [40]:
def logits_processor(prev_tok_ids, next_tok_logits):
    # example, zero everything but 'um'
    # next_tok_logits = next_tok_logits*0
    # next_tok_logits[3205] = 100

    # example, adds bias to 'um'
    next_tok_logits[3205] += 1
 
    return next_tok_logits

resp = llm(
    "this is um ",
    max_tokens=10,
    logprobs=5, # Show top 5 most likely tokens
    logits_processor=[logits_processor],
    temperature=0
)

Llama.generate: 4 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =     176.93 ms
llama_perf_context_print: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    10 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =     422.17 ms /    11 tokens


In [41]:
print(resp['choices'][0]['text']) # llm output
print()
print(resp['choices'][0]['logprobs']['tokens']) # llm output tokens
print()
for lps in resp['choices'][0]['logprobs']['top_logprobs']: # llm output top-N logprobs. This data could lend a good explanation of temperature
    print(lps)

100% a question for the experts here

['1', '0', '0', '%', ' a', ' question', ' for', ' the', ' experts', ' here']

{'1': np.float32(-1.6013106), '2': np.float32(-1.8831013), '3': np.float32(-2.6544409), 'ive': np.float32(-2.7861729), '0': np.float32(-2.8277216)}
{'0': np.float32(-1.4950409), '2': np.float32(-2.0479078), '1': np.float32(-2.1067858), '9': np.float32(-2.6830473), '3': np.float32(-2.7920685)}
{'0': np.float32(-1.2913281), ' minutes': np.float32(-2.54077), '1': np.float32(-2.883245), ' days': np.float32(-3.369475), '.': np.float32(-3.399469)}
{'%': np.float32(-0.4586866), ' percent': np.float32(-2.3217494), '0': np.float32(-2.5390441), '1': np.float32(-4.437709), ' times': np.float32(-4.6682396)}
{' a': np.float32(-2.4757462), ' my': np.float32(-2.5984821), ' true': np.float32(-2.6285858), ' not': np.float32(-3.194232), ' off': np.float32(-3.290863)}
{' question': np.float32(-3.0960052), ' r': np.float32(-3.2916553), ' copy': np.float32(-3.8266113), ' no': np.float32(-3.85